# Scraping Show Transcripts 
[ER](https://transcripts.foreverdreaming.org/viewforum.php?f=1035), [Scrubs](https://transcripts.foreverdreaming.org/viewforum.php?f=1027&sid=40edea8bd662f46aec12deaaca6c47ce), [House](https://clinic-duty.livejournal.com/12225.html), [Grey’s Anatomy](https://transcripts.foreverdreaming.org/viewforum.php?f=11), and [The Good Doctor](https://transcripts.foreverdreaming.org/viewforum.php?f=857)

In [18]:
from bs4 import BeautifulSoup

import requests
import pandas as pd
from IPython.display import clear_output
from tqdm import tqdm

In [31]:
show_id = 857
int_show_id = 6470478
num_eps = 80

## Get Links

In [32]:
a_tags = []
n = 0
while n < num_eps-1:
    response = requests.get("https://transcripts.foreverdreaming.org/viewforum.php?f="+str(show_id)+"&start="+str(n))
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    posts = document.find_all("div", "xbox-layout-forum")[1]
    a_tags = a_tags + posts.find_all("a", "topictitle")
    n = len(a_tags)
    print("Currently pulled "+str(n)+" tags")

Currently pulled 78 tags
Currently pulled 127 tags


## Extract Scripts

In [21]:
def extract_script(link):
    response = requests.get("https://transcripts.foreverdreaming.org/"+link[0:])
    html_string = response.text
    soup = BeautifulSoup(html_string, "html.parser")
    script = soup.find_all('div', 'content')[0]
    parsed_script = [string for string in script.stripped_strings]

    return parsed_script

In [22]:
def scrape_episodes(a_tags):
    links = [tag.get("href") for tag in a_tags]
    seasons = [int(tag.string[:2]) for tag in a_tags]
    episodes = [int(tag.string[3:5]) for tag in a_tags]
    scripts = []
    
    for link in tqdm(links):
        
        scripts.append(extract_script(link))

    dict = {"show_id": int_show_id, "season": seasons, "episode": episodes, "script": scripts}
    return pd.DataFrame(dict)


In [33]:
df = scrape_episodes(a_tags)

100%|██████████| 127/127 [00:42<00:00,  3.00it/s]


In [34]:
df.sort_values(by=['season', 'episode'], ignore_index=True, inplace=True)

## Add Scripts to Database

In [35]:
df.to_csv("./data/transcripts/good_doctor_transcripts.csv")